In [14]:
# Import Tools

import requests
from bs4 import BeautifulSoup
import pandas as pd
import random

In [15]:
tag_urls = {
    'Politics': 'https://medium.com/tag/politics/archive/{0}/{1:02d}/{2:02d}'
}

In [16]:
year = 2022
selected_days = [i for i in range(1, 48)] #Every day of the year

In [17]:
def convert_day(day):
    # if it is a leap year use month_days = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [18]:
def get_claps(claps_str):
    if (claps_str is None) or (claps_str == '') or (claps_str.split is None):
        return 0
    split = claps_str.split('K')
    claps = float(split[0])
    claps = int(claps*1000) if len(split) == 2 else int(claps)
    return claps

In [28]:
articles_data = []
article_id = 0
n = len(selected_days)
for d in selected_days:
    month, day = convert_day(d)
    date = '{0}-{1:02d}-{2:02d}'.format(year, month, day) 
    print(f'{date}')
    for tag, url in tag_urls.items(): 
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all(
            "div",
            class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        
        for article in articles:
            
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            title = title.contents[0]
            
            author = article.find_all("a")[0]['href'].split('?')[0].split('@')[1]
            author_url = article.find_all("a")[0]['href'].split('?')[0]
            
            subtitle = article.find("h4", class_="graf--subtitle")
            subtitle = subtitle.contents[0] if subtitle is not None else ''
            
            article_url = article.find_all("a")[3]['href'].split('?')[0]
            
            reading_time = article.find("span", class_="readingTime")
            reading_time = 0 if reading_time is None else int(reading_time['title'].split(' ')[0])
            
            responses = article.find_all("a")
            if len(responses) == 7:
                responses = responses[6].contents[0].split(' ')
                if len(responses) == 0:
                    responses = 0
                else:
                    responses = responses[0]
            else:
                responses = 0
                
            articles_data.append([article_url, title,
                         author, author_url,
                         subtitle, responses,
                         reading_time, tag, date])

2022-01-01
2022-01-02
2022-01-03
2022-01-04
2022-01-05
2022-01-06
2022-01-07
2022-01-08
2022-01-09
2022-01-10
2022-01-11
2022-01-12
2022-01-13
2022-01-14
2022-01-15
2022-01-16
2022-01-17
2022-01-18
2022-01-19
2022-01-20
2022-01-21
2022-01-22
2022-01-23
2022-01-24
2022-01-25
2022-01-26
2022-01-27
2022-01-28
2022-01-29
2022-01-30
2022-01-31
2022-02-01
2022-02-02
2022-02-03
2022-02-04
2022-02-05
2022-02-06
2022-02-07
2022-02-08
2022-02-09
2022-02-10
2022-02-11
2022-02-12
2022-02-13
2022-02-14
2022-02-15
2022-02-16
2022-02-17


In [21]:
# Transform article data into panda dataframe.
medium_df = pd.DataFrame(articles_data, columns=[
    'url', 'title', 'author', 'author_page',
    'subtitle', 'responses', 'reading_time',
    'tag', 'date'])

In [22]:
medium_df.shape

(7041, 9)

In [23]:
medium_df = medium_df.drop_duplicates(subset=['url', 'title'], keep='first')

In [24]:
medium_df.shape

(7041, 9)

In [26]:
medium_df

,url,title,author,author_page,subtitle,responses,reading_time,tag,date
0,https://aninjusticemag.com/a-fair-media-would-...,A Fair Media Would Emphasize that Republicans ...,walterrhein,https://aninjusticemag.com/@walterrhein,Quit talking about “the…,15,6,Politics,2022-01-01
1,https://eand.co/what-kind-of-year-is-2022-goin...,What Kind of Year is 2022 Going to Be?,umairh,https://eand.co/@umairh,"Don’t Click This, Because I’m Going to Ruin Yo...",44,11,Politics,2022-01-01
2,https://medium.com/@timjwise/lets-go-america-o...,"Let’s Go, America (or, Why COVID is Forever Now)",timjwise,https://medium.com/@timjwise,"When you kill the idea of the public good, pri...",0,8,Politics,2022-01-01
3,https://medium.com/@rayfield_98709/mass-format...,Mass Formation Psychosis,rayfield_98709,https://medium.com/@rayfield_98709,,0,1,Politics,2022-01-01
4,https://medium.com/politically-speaking/should...,Should Democrats Who Move to Red States Not be...,carenawhite,https://medium.com/@carenawhite,,18,4,Politics,2022-01-01
...,...,...,...,...,...,...,...,...,...
7036,https://medium.com/@akshayprasad891/lebanons-c...,Lebanon’s Central Bank Chief Is Working From H...,akshayprasad891,https://medium.com/@akshayprasad891,,0,2,Politics,2022-02-17
7037,https://medium.com/memes-of-note/big-tech-will...,Big Tech will censor anything they don’t like…,showkster,https://medium.com/@showkster,,0,1,Politics,2022-02-17
7038,https://medium.com/@makemyassignments/politica...,[Political Science Research assignment topics],makemyassignments,https://medium.com/@makemyassignments,,0,2,Politics,2022-02-17
7039,https://medium.com/memes-of-note/those-people-...,Those people … and their media …,showkster,https://medium.com/@showkster,,0,1,Politics,2022-02-17


In [27]:
medium_df.to_csv('medium-data-politics-1-2022.csv', index=True)